In [ ]:
import numpy as np
# Load pandas
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score

In [ ]:
filename = "/Users/maria/Downloads/train_2008.csv"
filename_test = "/Users/maria/Downloads/test_2008.csv"
data = np.loadtxt(filename, skiprows=1, delimiter=',')

test_data = np.loadtxt(filename_test, skiprows=1, delimiter=',')

# Set random seed
np.random.seed(0)

In [ ]:
X_train = (np.asarray(data))[:, 3:382].astype(np.float)
y_train = (np.asarray(data))[:, 382].astype(np.float)

df = pd.DataFrame(data)

# remove first three columns

# rename 382th column to did_vote
df = df.rename(columns={ df.columns[382]: "did_vote" })
df.head()

In [ ]:
X_test = (np.asarray(test_data))[:, 3:382].astype(np.float)
y_test = (np.asarray(test_data))[:, 381].astype(np.float)

In [ ]:
rf = RandomForestRegressor(max_features=2, min_samples_split=4, n_estimators=50, min_samples_leaf=2, max_depth=9)

gb = GradientBoostingRegressor(loss='quantile', learning_rate=0.0001, n_estimators=50, max_features='log2', min_samples_split=2, max_depth=1)

ada_tree_backing = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3)

ab = AdaBoostRegressor(ada_tree_backing, learning_rate=0.1, loss='square', n_estimators=1000)

# validate.cross_v_scores([rf, gb, ab], X_train, y_train)

In [ ]:
rf_model = rf.fit(X_train, y_train)

In [ ]:
y_predict = rf_model.predict(X_train)

In [ ]:
roc_auc_score(y_train, y_predict)

In [ ]:
y_predict_test = rf_model.predict(X_test)

In [ ]:
print (ids.shape)
print (y_predict_test.astype(float).shape)
y_predict_test = y_predict_test.reshape(16000, 1)

In [ ]:
# output to file
ids = test_data[:,:1]
ids = ids.astype(int)
predictions = y_predict_test.astype(float)
out = np.concatenate((ids, predictions), axis=1)

np.savetxt('rf_submission.csv', out, delimiter=',', fmt='%1.10f', header = 'id,target')
# NOTE: REMEMBER TO REMOVE THE # SYMBOL FROM THE ID HEADER BEFORE SUBMITTING

In [ ]:
gb_model = gb.fit(X_train, y_train)
y_predict_gb = gb_model.predict(X_train)

In [ ]:
roc_auc_score(y_train, y_predict_gb)

In [ ]:
ab_model = ab.fit(X_train, y_train)
y_predict_ab = gb_model.predict(X_train)

In [ ]:
roc_auc_score(y_train, y_predict_ab)

In [ ]:
ab = AdaBoostRegressor(base_estimator=rf_model, learning_rate=0.1, loss='square', n_estimators=1000)

In [ ]:
ab2_model = ab.fit(X_train, y_train)
y_predict_ab2 = gb_model.predict(X_train)

In [ ]:
roc_auc_score(y_train, y_predict_ab2)

In [ ]:
from sklearn import svm
X = X_train
y = y_train
clf = svm.SVR()
clf.fit(X, y) 
# SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
#     gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
#     tol=0.001, verbose=False)
clf.predict(X_train)

In [5]:
import csv
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from keras.regularizers import l1_l2
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

filename = "/Users/maria/Downloads/train_2008.csv"

# load data from CSV files
# extract data into data_array by row, first row is column labels
with open(filename, 'r') as srcfile:
    data_iter = csv.reader(srcfile, quotechar = '"')
    data = [data for data in data_iter]
    data = data[1:]
    print(data)

def normalize(inputs):
    print ("Normalizing inputs...")
    scaler = StandardScaler().fit(inputs)
    norm_inputs = scaler.transform(inputs)
    print ("Done normalizing. ")
    return (scaler, norm_inputs)

# note: two columns removed for x
X_train = (np.asarray(data))[:, 3:382].astype(np.float)
y_train = (np.asarray(data))[:, 382].astype(np.float)

# print(np.shape(X_train))
# print(y_train)

norm_packet = normalize(X_train)
norm_train = norm_packet[1]
scaler = norm_packet[0]
norm_train = np.array(norm_train)

## In your homework you should transform each input data point
## into a single vector here and should transform the
## labels into a one hot vector using np_utils.to_categorical

# our results fall into two categories
y_train_hot = np.empty([0, 2])
X_train_hot = np.array(X_train)

y_train_hot -= 1

for i in range(0, y_train.size):
    y_train_hot = np.vstack((y_train_hot, to_categorical(y_train[i], 2)))

# create our model
model = Sequential()
model.add(Dense(1000, input_shape=(379,), init='normal', activation='sigmoid'))
model.add(Dense(1000, init='normal', activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1000, init='normal', activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1000, init='normal', activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
fit = model.fit(X_train_hot, y_train_hot, batch_size=1000, nb_epoch=30,verbose=1)
score = model.evaluate(X_train_hot, y_train_hot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizing inputs...
Done normalizing. 


/Users/maria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, input_shape=(379,), activation="sigmoid", kernel_initializer="normal")`
/Users/maria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, activation="sigmoid", kernel_initializer="normal")`
/Users/maria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, activation="sigmoid", kernel_initializer="normal")`
/Users/maria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1000, activation="sigmoid", kernel_initializer="normal")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1000)              380000    
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000)              0         
__________

/Users/maria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:67: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/30
64667/64667 [==============================] - 45s 701us/step - loss: 4.0572 - acc: 0.7432
Epoch 2/30
64667/64667 [==============================] - 33s 514us/step - loss: 4.1163 - acc: 0.7446
Epoch 3/30
64667/64667 [==============================] - 33s 518us/step - loss: 4.1163 - acc: 0.7446
Epoch 4/30
64667/64667 [==============================] - 33s 509us/step - loss: 4.1163 - acc: 0.7446
Epoch 5/30
64667/64667 [==============================] - 33s 507us/step - loss: 4.1163 - acc: 0.7446
Epoch 6/30
64667/64667 [==============================] - 33s 506us/step - loss: 4.1163 - acc: 0.7446
Epoch 7/30
64667/64667 [==============================] - 33s 506us/step - loss: 4.1163 - acc: 0.7446
Epoch 8/30
64667/64667 [==============================] - 33s 504us/step - loss: 4.1163 - acc: 0.7446
Epoch 9/30
64667/64667 [==============================] - 34s 519us/step - loss: 4.1163 - acc: 0.7446
Epoch 10/30
64667/64667 [==============================] - 35s 547us/step - loss: 